# Test pdb 1nk3

In [7]:
# code modified based on Xingcheng's code
import sys
import numpy as np
import pandas as pd
import simtk.openmm
import simtk.unit as unit
import os
import glob
import shutil
import time
import MDAnalysis as mda
import math
import mdtraj
pd.set_option("display.precision", 10)

ca_sbm_3spn_openmm_path = '/Users/xl23/GitHub/CA_SBM_3SPN2C_OPENMM'
sys.path.insert(0, ca_sbm_3spn_openmm_path)

import openSMOG3SPN2.open3SPN2.ff3SPN2 as ff3SPN2
import openSMOG3SPN2.calphaSMOG.ffCalpha as ffCalpha
import openSMOG3SPN2.openFiber as openFiber

output_dir = f'{ca_sbm_3spn_openmm_path}/jupyter-notebook/output-files/1nk3'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
os.chdir(output_dir)


In [8]:
# load 1nk3 pdb file
pdb_path = f'{ca_sbm_3spn_openmm_path}/data/1nk3/atomistic.pdb'
fix = ff3SPN2.fixPDB(pdb_path)
complex_table = ff3SPN2.pdb2table(fix)
cg_protein = ffCalpha.Protein.CoarseGrain(complex_table)
cg_dna = ff3SPN2.DNA.CoarseGrain(complex_table)
protein = ffCalpha.Protein.fromCoarsePandasDataFrame(df=cg_protein)
smog_files_dir = f'{ca_sbm_3spn_openmm_path}/data/1nk3'
protein.add_smog_bonds(f'{smog_files_dir}/bonds.dat')
protein.add_smog_angles(f'{smog_files_dir}/angles.dat')
protein.add_smog_dihedrals(f'{smog_files_dir}/dihedrals.dat')
protein.add_smog_native_pairs(f'{smog_files_dir}/pairs.dat')
dna = ff3SPN2.DNA.fromCoarsePandasDataFrame(df=cg_dna, dna_type='B_curved')
protein_dna = openFiber.SMOG3SPN2()
protein_dna.add_protein_dna_object(protein)
protein_dna.add_protein_dna_object(dna)
atoms_to_pdb = protein_dna.atoms.copy()

chainID_dict = {1:'A', 2:'B', 3:'C'}
original_chainID = atoms_to_pdb['chainID']
new_chainID = []
for i in original_chainID:
    new_chainID.append(chainID_dict[int(i)])
atoms_to_pdb['chainID'] = new_chainID
protein_dna.pdb = 'cg_1nk3.pdb'
ffCalpha.writePDB(atoms_to_pdb, protein_dna.pdb)

protein_dna.periodic = True
protein_dna.smog_energy_scale = 3.75
protein_dna.temperature = 300*unit.kelvin
protein_dna.salt_concentration = 150*unit.millimolar
protein_dna.parse_smog_exclusions(exclude_native_pairs=False)
protein_dna.smog_exclusions.to_csv('smog_exclusions.csv', index=False)
protein_dna.create_system()
protein_dna.add_forces()



Set smog bonded energy scale as None
use the new set of base step geometry parameters
the DNA sequence is W-C paired!
use the sequence of the first ssDNA as the input sequence for x3dna
Combine smog and dna exclusions
Adding force SMOGBond takes 0.0058002471923828125 seconds
Adding force SMOGAngle takes 0.005472898483276367 seconds
Adding force SMOGDihedral takes 0.012825965881347656 seconds
Adding force SMOGNativePair takes 0.011469125747680664 seconds
Adding force DNABond takes 0.01340484619140625 seconds
Adding force DNAAngle takes 0.016267061233520508 seconds
Adding force DNAStacking takes 0.005185127258300781 seconds
Adding force DNADihedral takes 0.01698899269104004 seconds
Adding force DNABasePair takes 0.02295708656311035 seconds
Adding force DNACrossStacking takes 0.08803796768188477 seconds
Adding force AllVanderWaals takes 0.11543083190917969 seconds
For electrostatic interactions, set monovalent salt concentration as 150 mM
For electrostatic interactions, set temperature as

In [9]:
protein_dna.top = simtk.openmm.app.PDBFile(protein_dna.pdb).getTopology()
protein_dna.init_coord = simtk.openmm.app.PDBFile(protein_dna.pdb).getPositions(asNumpy=True)
protein_dna.set_simulation(timestep=5*unit.femtosecond, platform_name='CPU')

columns = ['overall', 'protein bond', 'protein angle', 'protein dihedral', 'native pair', 'DNA bond', 'DNA angle', 'stacking', 'DNA dihedral', 'base pair', 'cross stacking', 'Van der Waals', 'electrostatic']
df_energy = pd.DataFrame(columns=columns)
row = []

#Set initial positions
protein_dna.simulation.context.setPositions(protein_dna.init_coord)

state = protein_dna.simulation.context.getState(getEnergy=True)
energy = state.getPotentialEnergy().value_in_unit(unit.kilocalorie_per_mole)
row.append(energy)
for j in range(1, 13):
    state = protein_dna.simulation.context.getState(getEnergy=True, groups={j})
    energy = state.getPotentialEnergy().value_in_unit(unit.kilocalorie_per_mole)
    row.append(energy)
row = np.array(row)
df_energy.loc[len(df_energy.index)] = row
df_energy = df_energy.round(6)
#print(df_energy)

Use NoseHooverIntegrator
Use platform CPU


In [10]:
# load dcd coordinates
dcd1 = f'{ca_sbm_3spn_openmm_path}/data/1nk3/initial.dcd'
dcd2 = f'{ca_sbm_3spn_openmm_path}/data/1nk3/perturbed.dcd'
for each_dcd in [dcd2]:
    with mdtraj.formats.DCDTrajectoryFile(each_dcd, 'r') as f:
        xyz, cell_lengths, cell_angles = f.read()
    xyz /= 10 # convert unit to nm
    protein_dna.simulation.context.setPositions(xyz[0])
    row = []
    state = protein_dna.simulation.context.getState(getEnergy=True)
    energy = state.getPotentialEnergy().value_in_unit(unit.kilocalorie_per_mole)
    row.append(energy)
    for j in range(1, 13):
        state = protein_dna.simulation.context.getState(getEnergy=True, groups={j})
        energy = state.getPotentialEnergy().value_in_unit(unit.kilocalorie_per_mole)
        row.append(energy)
    row = np.array(row)
    df_energy.loc[len(df_energy.index)] = row
    df_energy = df_energy.round(6)
print(df_energy)
    

        overall  protein bond  protein angle  protein dihedral  native pair  \
0   1123.274817      0.000000       0.000000          0.000000  -104.867065   
1  22690.253334   3028.383273      79.443059         11.625255   -57.225121   

      DNA bond    DNA angle      stacking  DNA dihedral   base pair  \
0    57.083183   340.233200    908.572116    -59.381475    3.761660   
1  1711.467230  1102.903576  16166.192269    -48.120850  711.320178   

   cross stacking  Van der Waals  electrostatic  
0       -7.399611      -6.248051      -8.479139  
1       -3.488199      -4.081793      -8.165544  


In [11]:
protein_dna.simulation.context.setPositions(protein_dna.init_coord)
protein_dna.simulation.minimizeEnergy()
protein_dna.simulation.context.setVelocitiesToTemperature(protein_dna.temperature)
protein_dna.add_reporters(10, report_dcd=False)
protein_dna.simulation.step(100)

#"Step","Time (ps)","Potential Energy (kJ/mole)","Kinetic Energy (kJ/mole)","Total Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
10,0.049999999999999996,-1363.2328770535821,462.8456124578946,-900.3872645956875,236.38017762846667,0
20,0.10000000000000002,-1325.079231884133,423.2206726288563,-901.8585592552766,216.14329936237428,19.9
30,0.15000000000000005,-1288.4573731707762,381.4714386176549,-906.9859345531213,194.8215214610704,20
40,0.2000000000000001,-1253.3880659047975,349.9073719575439,-903.4806939472536,178.70141686685747,19.8
50,0.2500000000000001,-1252.509081194196,348.8379380847106,-903.6711431094853,178.15524561230066,19.8
60,0.30000000000000016,-1273.1343238530817,373.82039642664324,-899.3139274264385,190.91405282903543,19.7
70,0.3500000000000002,-1268.4535112641058,370.55056755100827,-897.9029437130976,189.2441164406731,19.6
80,0.40000000000000024,-1286.8094410601075,394.0336892913152,-892.7757517687922,201.23719650632813,19.6
90,0.4500000000000003,-1298.8238614179968